# 数据库操作封装类

负责与sqlite数据库交互，暴露以下API：

- 添加一个物品
- 删除一个物品
- 查询所有物品
- 查询等级大于某个值的所有物品
- 添加一个配方
- 删除一个配方
- 根据主产品、是否高效查询一个配方
- 查询所有配方

In [20]:
import pandas as pd

from dataclasses import dataclass
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Boolean

# 数据库连接，echo=True将会打印日志
engine = create_engine(
    "sqlite:///dyson.db?check_same_thread=False", echo=False)
# 数据库会话
session = sessionmaker(bind=engine)()

Base = declarative_base()


@dataclass
class Item(Base):
    """
    物品类，包括所有材料及产物。
    """
    __tablename__ = 't_item'
    id: int = Column(Integer, primary_key=True)
    name: str = Column(String(20), unique=True)  # 物品名唯一
    level: int = Column(Integer)


@dataclass
class Formula(Base):
    """
    配方类
    dataclass必须显式标记属性的类型
    """
    __tablename__ = 't_formula'
    id: int = Column(Integer, primary_key=True)  # 主键
    name: str = Column(String(20), unique=True)  # 配方名唯一
    work_time: int = Column(Integer)  # 生产时间，单位为秒
    # 材料，形如1-2,3-4，意为需要id为1的物品2个，id为3的物品4个，依此类推
    material: str = Column(String(200))
    # 产品，形如1-2,3-4，意为产出id为1的物品2个，id为3的物品4个，依此类推，排在最前面的是主产品
    product: str = Column(String(200))
    # 制造于哪种设备，如果是制造台则可以加速
#     madein:str=Column(String(20))
    is_efficient: bool = Column(Boolean, default=False)  # 是否是高效配方，默认false
    is_adjustable: bool = Column(
        Boolean, default=False)  # 可调节速度，如果为true，量化时需要给定系数
    is_enabled: bool = Column(
        Boolean, default=False)  # 配方是否启用


Base.metadata.create_all(engine)  # 创建所有表


def save_item(name, level):
    """
    新增一个物品
    """
    try:
        item = Item(name=name, level=level)
        session.add(item)
        session.commit()
    except:
        session.rollback()


def remove_item(name):
    """
    根据名称删除一个物品
    """
    try:
        item = session.query(Item).filter_by(name=name.strip()).first()
        session.delete(item)
        session.commit()
    except:
        session.rollback()


def query_items(name='', level=0):
    """
    根据名称模糊查询，并指定等级大于等于某个值
    不传任何参数相当于查询全部
    """
    return session.query(Item).filter(
        Item.name.like('%'+name+'%'), Item.level >= level).all()


def query_one_item(name=''):
    """
    根据名称精确查询
    """
    return session.query(Item).filter_by(name=name).first()


def save_formula(name, work_time, material, product, is_efficient=False, is_adjustable=False, is_enabled=False):
    """
    新增一个配方
    """
    try:
        f = Formula(name=name, work_time=work_time, material=material,
                    product=product, is_efficient=is_efficient, is_adjustable=is_adjustable, is_enabled=is_enabled)
        session.add(f)
        session.commit()
    except:
        session.rollback()


def remove_formula(name):
    """
    删除一个配方
    """
    try:
        f = session.query(Formula).filter_by(name=name.strip()).first()
        session.delete(f)
        session.commit()
    except:
        session.rollback()


def set_formula_enabled(name, enabled):
    """
    启用或关闭配方
    """
    try:
        f = session.query(Formula).filter_by(
            name=name.strip()).first()
        f.is_enabled = enabled
        session.commit()
    except:
        session.rollback()


def query_formula_by_product_name(name, is_efficient=False):
    """
    根据主产品名称和是否是高效查询一个配方
    主要用于递归中查询下级配方
    """
    return session.query(Formula).filter(Formula.product.like(name+'%'), Formula.is_efficient == is_efficient).first()


def query_formula_by_name(name='', enabled=True):
    """
    根据名称模糊查询配方，不传任何参数相当于查询全部
    """
    return session.query(Formula).filter(Formula.name.like('%'+name+'%'), Formula.is_enabled == enabled).all()


def query_formula_data(name=''):
    sql = f"""
            SELECT * FROM t_formula WHERE NAME LIKE '%{name}%'
            """
    return pd.read_sql(sql, engine)


# __all__ = [save_item, remove_item, query_item, save_formula,
#            remove_formula, query_formula_by_id, query_formula_by_name,query_formula_data]

# f=query_formula_by_name()
# f

# f = query_item()
# f

# set_formula_enabled('磁线圈', True)

In [10]:
#     session.execute("update t_formula set is_enabled=0 where id=13")
#     session.commit()